In [1]:
import os
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'
import torch
import json
import requests
from pathlib import Path
from pprint import pprint #Pretty Print to print better json
from PyPDF2 import PdfReader
from openai import OpenAI
import langchain_community
from langchain.document_loaders import PyPDFLoader 
from langchain.text_splitter import RecursiveJsonSplitter # Recursive Json splitter to spilit data.
from langchain_community.document_loaders import UnstructuredURLLoader #URL loaders 
from langchain_community.document_loaders import AsyncHtmlLoader #AsyncHTML Loader to load HTML into data.
from langchain.text_splitter import CharacterTextSplitter # Character Text Spilitter to split text in chunks.
from langchain_community.document_loaders import JSONLoader #Load Json files into the code using JSONLoadder.
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings # Embeddings from Hugging face Hub
from sentence_transformers import SentenceTransformer # Scentence Transformers to transfromation model.
import faiss # Facebook research model
import numpy as np # Numpy array
from transformers import AutoTokenizer
from adapters import AutoAdapterModel  
from langchain.vectorstores import FAISS #Langchain Vector store 
from transformers import AutoTokenizer, AutoModelForCausalLM #LLM (Llama2)
from transformers import AutoTokenizer, AutoModel





# Load the Json file using json loader.
file_path = './final_data.json'
data = json.loads(Path(file_path).read_text())



texts = []
for paper in data:
    text = f"{paper['Title']} {paper.get('Abstract', '')} {paper['Authors']} {paper.get('Author(s) ID','')}{paper.get('References', '')}"
    texts.append(text)



texts = texts[:100]




model_name = "allenai/specter2_base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)



# model.load_adapter("allenai/specter2", source="hf", load_as="specter2", set_active=True)







# model = SentenceTransformer("allenai-specter2")
# tokenizer = AutoTokenizer.from_pretrained("allenai/specter2_base")
# model = AutoAdapterModel.from_pretrained("allenai/specter2_base")


# Load model directly


# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")


inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


# Check if GPU is available
if torch.cuda.is_available():
    print("Running on GPU MODE")
    inputs = {key: tensor.cuda() for key, tensor in inputs.items()}
    model.cuda()
else:
    print("Running on CPU")

# Generate the model's output
with torch.no_grad():
    outputs = model(**inputs)

# Retrieve embeddings from the last hidden state
embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Convert to numpy
print("This part is also working")

# Add to FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
embeddings = embeddings.astype('float32')
index.add(embeddings)



print("FAISS index built successfully with new embeddings!")




def query_index(query, index, data, model, tokenizer):
    # Tokenize and encode the query
    query_input = tokenizer(query, padding=True, truncation=True, return_tensors="pt", max_length=512)

    # Move to GPU if available
    if torch.cuda.is_available():
        query_input = {key: tensor.cuda() for key, tensor in query_input.items()}

    with torch.no_grad():
        query_output = model(**query_input)

    query_embedding = query_output.last_hidden_state[:, 0, :].cpu().numpy().astype('float32')

    # Perform similarity search
    distances, indices = index.search(query_embedding, 5)  # Retrieve top 5 results

    # Display results
    print("Query Results:")
    print("Indices of matching documents:", indices[0])
    for dist, idx in zip(distances[0], indices[0]):
        
        paper = data[idx]
        print(f"Title: {paper['Title']}")
        print(f"Abstract: {paper.get('Abstract', '')}")
        print(f"Authors: {paper['Authors']}")
        print(f"Distance: {dist}")
        print("\n")



user_query = input("Enter your query: ")
query_index(user_query, index, data, model, tokenizer)


/Users/shishirgupta/Car_Price_prediction /Research_data_RAG/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


RuntimeError: Failed to import adapters.models.auto because of the following error (look up to see its traceback):
cannot import name 'url_to_filename' from 'huggingface_hub.file_download' (/Users/shishirgupta/Car_Price_prediction /Research_data_RAG/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py)